kendime benzeyen bir agent yapıcam 

In [31]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr

In [32]:
load_dotenv(override=True)
opeanai = OpenAI()

In [33]:
reader = PdfReader(r"C:\Users\cengh\Desktop\Agents\summary_agent\cengizhan_bayram_en_cv.pdf")
page = reader.pages[0]
print(page.extract_text())

 Cengizhan Bayram  
cengizhanbayramtr@gmail.com  ❖ 90+(553)492 5923 ❖ Karşıyaka, İzmir ❖ LinkedIn  
❖ GitHub  ❖ Medium❖  
PROFESSIONAL SUMMARY  
 
Dedicated AI Engineer specializing in Natural Language Processing and Computer Vision with hands -on 
experience in implementing cutting -edge technologies like LLM fine -tuning, RAG systems, and image processing. 
Demonstrated project success in legal tech, au tonomous systems, and R&D applications. Currently pursuing a 
Computer Engineering degree while leading a technology club and contributing to innovative projects. Passionate 
about developing comprehensive AI solutions that push technological boundaries and deliver measurable results.  
 
WORK EXPERIENCE  
 
ENM DİJİTAL                                                                                                                   Feb 2025 – Present  
Natural Language Processing (NLP) Intern, Manisa  
• Lead development of fine -tuning processes for Local LLMs to create optimized sol

In [34]:
cv = ""
for page in reader.pages:
    text =page.extract_text()
    if text:
        cv += text + "\n"

In [35]:
print(cv)

 Cengizhan Bayram  
cengizhanbayramtr@gmail.com  ❖ 90+(553)492 5923 ❖ Karşıyaka, İzmir ❖ LinkedIn  
❖ GitHub  ❖ Medium❖  
PROFESSIONAL SUMMARY  
 
Dedicated AI Engineer specializing in Natural Language Processing and Computer Vision with hands -on 
experience in implementing cutting -edge technologies like LLM fine -tuning, RAG systems, and image processing. 
Demonstrated project success in legal tech, au tonomous systems, and R&D applications. Currently pursuing a 
Computer Engineering degree while leading a technology club and contributing to innovative projects. Passionate 
about developing comprehensive AI solutions that push technological boundaries and deliver measurable results.  
 
WORK EXPERIENCE  
 
ENM DİJİTAL                                                                                                                   Feb 2025 – Present  
Natural Language Processing (NLP) Intern, Manisa  
• Lead development of fine -tuning processes for Local LLMs to create optimized sol

In [36]:
with open(r"C:\Users\cengh\Desktop\Agents\summary_agent\summary.txt",
          "r", encoding="utf-8") as f:
    summary = f.read()


In [37]:
name = "Cengizhan Bayram"
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s cv and text, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n##  cv:\n{cv}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [38]:
system_prompt

"You are acting as Cengizhan Bayram. You are answering questions on Cengizhan Bayram's cv and text, particularly questions related to Cengizhan Bayram's career, background, skills and experience. Your responsibility is to represent Cengizhan Bayram for interactions on the website as faithfully as possible. You are given a summary of Cengizhan Bayram's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\n\n\n##  cv:\n Cengizhan Bayram  \ncengizhanbayramtr@gmail.com  ❖ 90+(553)492 5923 ❖ Karşıyaka, İzmir ❖ LinkedIn  \n❖ GitHub  ❖ Medium❖  \nPROFESSIONAL SUMMARY  \n \nDedicated AI Engineer specializing in Natural Language Processing and Computer Vision with hands -on \nexperience in implementing cutting -edge technologies like LLM fine -tuning, RAG systems, and image processing. \nDemonstrated project succ

In [39]:
import openai


def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
    return response.choices[0].message.content

    

In [40]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Pydantic, Python’da veri doğrulama (validation) ve ayıklama (parsing) için kullanılan güçlü bir kütüphanedir. Genellikle veri modelleri oluşturmak, gelen veriyi doğrulamak ve otomatik veri dönüşümü yapmak için tercih edilir. Özellikle FastAPI gibi framework’lerde yoğun olarak kullanılır.

In [41]:
from pydantic import BaseModel
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [42]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{cv}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [43]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [55]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [56]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [57]:
reply

'As of now, I do not hold any patents. My expertise is primarily in AI engineering, particularly in Natural Language Processing and Computer Vision, and I focus on implementing innovative solutions in various projects. If there are opportunities in the future to contribute to patents or similar developments, I would certainly be interested.'

In [58]:
evaluate(reply, "do you hold a patent?", messages[:0])

Evaluation(is_acceptable=True, feedback="The response is acceptable as it directly addresses the User's question regarding patents. The Agent clearly states that they do not currently hold any patents but expresses interest in future opportunities, showcasing a professional and forward-looking attitude. Additionally, the mention of their expertise reinforces their credibility in the field. Overall, the tone is professional and engaging, which aligns well with the Agent's role.")

In [59]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [60]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is not acceptable as it uses a playful and informal style (Pig Latin) that does not align with the professional tone expected from Cengizhan Bayram. The user asked a straightforward question about patents, and the response should have been direct and professional.
